In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from mgtwr.function import _compute_betas_gwr
from mgtwr.kernel import GWRKernel  # Use GWRKernel from kernel.py for kernel operations
from mgtwr.model import GWR  # Import GWR class from model.py
from mgtwr.sel import SearchGWRParameter
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error, mean_absolute_error

In [2]:
ewhp_calib = pd.read_csv("ewhp_calib.csv")
coords_C = ewhp_calib[["Easting", "Northing"]]
X_C = ewhp_calib[["FlrArea"]]
y_C = ewhp_calib[["PurPrice"]]

# Step 2: Calibrate the GWR model on the calibration dataset
bandwidth_selector = SearchGWRParameter(
    coords=coords_C, X=X_C, y=y_C, kernel="bisquare", fixed=False, thread=8
)  # Selecting bandwidth
calibration_bw = bandwidth_selector.search(
    criterion="CV", verbose=False, time_cost=False
)
print("Optimal calibration bandwidth:", calibration_bw)

# Fit the GWR model using the calibration data and the selected bandwidth
gwr_model = GWR(
    coords=coords_C,
    X=X_C,
    y=y_C,
    bw=calibration_bw,
    kernel="bisquare",
    fixed=False,
    thread=8,
)
gwr_results = gwr_model.fit()
print(gwr_results.R2, gwr_results.adj_R2)

Optimal calibration bandwidth: 51.0
0.6446328030581983 0.6006787132086768


In [3]:
ewhp_valid = pd.read_csv("ewhp_valid.csv")
coords_V = ewhp_valid[["Easting", "Northing"]]
X_V = ewhp_valid[["FlrArea"]]
y_V = ewhp_valid[["PurPrice"]]

# Step 3: Predict values for the validation dataset
y_pred = []  # List to store predictions

# Add an intercept column to calibration and validation datasets
X_C = np.hstack([np.ones((X_C.shape[0], 1)), X_C])  # Add intercept to calibration data
X_V = np.hstack([np.ones((X_V.shape[0], 1)), X_V])  # Add intercept to validation data

# Initialize the GWRKernel object with calibration coordinates
gwr_kernel = GWRKernel(
    coords=coords_C, bw=calibration_bw, fixed=False, function="bisquare"
)

# Iterate through each validation point
for coord_V, x_V in zip(coords_V.to_numpy(), X_V[:]):
    # Calculate distances from the validation point to all calibration points
    distances = cdist([coord_V], coords_C).reshape(-1)  # Distance to calibration points

    # Use the kernel object to calculate weights for this validation point
    weights = gwr_kernel.cal_kernel(distance=distances).reshape(
        -1, 1
    )  # Reshape weights for compatibility

    # Use `_compute_betas_gwr` to perform weighted least squares for the validation point
    betas, _ = _compute_betas_gwr(y=y_C, x=X_C, wi=weights)

    # Predict the dependent variable for the validation point
    y_V_pred = np.dot(
        x_V, betas
    )  # Dot product of coefficients with independent variables
    y_pred.append(y_V_pred)

# Step 4: Validate the predictions
y_pred = np.array(y_pred)
rmse = root_mean_squared_error(y_true=y_V, y_pred=y_pred)
mape = mean_absolute_percentage_error(y_true=y_V, y_pred=y_pred)
mae = mean_absolute_error(y_true=y_V, y_pred=y_pred)
print(f"RMSE = {rmse}")
print(f"MAE = {mae}")
print(f"MAPE = {mape}")

RMSE = 22804.551799446665
MAE = 15770.333702734004
MAPE = 0.276225524515927
